<a href="https://colab.research.google.com/github/AstxMargaryan/Data_Science/blob/main/Homework1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
!pip install duckdb

In [4]:
import duckdb

In [5]:
db ="/content/drive/MyDrive/HW1/db/transaction.duckdb"
con = duckdb.connect(db)

In [6]:
con.execute('''
CREATE TABLE transactions AS
SELECT *
FROM read_csv_auto('/content/drive/MyDrive/HW1/data_file/2019-Oct.csv.gz');
''')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [7]:
con.execute("SELECT * FROM transactions LIMIT 5").df()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00,view,44600062,2103807459595387724,None,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01,view,17200506,2053013559792632471,furniture.living_room.sofa,None,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


1.How many users have interacted with the eCommerce website? How many products have been viewed by those users?

In [43]:
con.execute("SELECT COUNT(DISTINCT user_id) AS num_of_user_interacted FROM transactions").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,num_of_user_interacted
0,3022290


In [44]:
con.execute(
'''
  SELECT
      COUNT(DISTINCT product_id) AS num_viewed_products
  FROM transactions
  WHERE event_type ='view'
''').df()

,num_viewed_products
0,166794


2.What were the top 5 most viewed product in October 2019?

In [26]:
con.execute('''
  SELECT
    product_id,
    COUNT(*) as view_count
  FROM transactions
  WHERE event_type = 'view'
  GROUP BY product_id
  ORDER BY view_count DESC
  LIMIT 5
''').df()

,product_id,view_count
0,1004856,419287
1,1004767,378777
2,1005115,327715
3,1004249,207422
4,1004833,203018


3.On average, how many items were added to cart?
  
  1 By user

  2 By user session

  3 Looking at the results of parts 1 & 2 of Question # 3, what can you say about the average number of sessions per user?

  4 Can you back your conclusion with data (using SQL)?

In [62]:
avg_cart_per_user = con.execute('''
WITH per_user AS(
  SELECT
    user_id,
    COUNT(*) AS cart_items
  FROM transactions
  WHERE event_type = 'cart'
  GROUP BY user_id
  )
SELECT AVG(cart_items) AS avg_cart_items_per_user
FROM per_user;
''').df()

In [63]:
avg_cart_per_user

,avg_cart_items_per_user
0,2.748351


In [64]:
avg_cart_per_session =con.execute('''
WITH per_session AS(
  SELECT
    user_session,
    COUNT(*) AS cart_items
  FROM transactions
  WHERE event_type = 'cart'
  GROUP BY user_session
  )
SELECT AVG(cart_items) AS avg_cart_items_per_session
FROM per_session;
''').df()

In [66]:
avg_cart_per_session

,avg_cart_items_per_session
0,1.61668


In [67]:
avg_session_per_user =  avg_cart_per_user['avg_cart_items_per_user'] /avg_cart_per_session['avg_cart_items_per_session']

In [68]:
avg_session_per_user

,0
0,1.699997


In [69]:
con.execute('''
WITH session_per_user AS (
  SELECT
    user_id,
    COUNT(DISTINCT user_session) AS num_sessions
  FROM transactions
  WHERE event_type = 'cart'
  GROUP BY user_id

)
SELECT AVG(num_sessions) AS avg_session_per_user
FROM session_per_user;
''').df()

,avg_session_per_user
0,1.699994


4. On average, how long do users take to visit the website again?

In [94]:
avg_time_between_visits=con.execute('''
WITH visits AS (
    SELECT
        user_id,
        CAST(event_time AS TIMESTAMP) AS event_time,
        LEAD(CAST(event_time AS TIMESTAMP)) OVER (
            PARTITION BY user_id
            ORDER BY event_time
        ) AS next_visit
    FROM transactions
),

diffs AS (
    SELECT
        user_id,
        (next_visit - event_time) AS time_diff
    FROM visits
    WHERE next_visit IS NOT NULL
)

SELECT
    AVG(time_diff) AS avg_time_between_visits
FROM diffs;
''').df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [95]:
hours = avg_time_between_visits['avg_time_between_visits'][0].total_seconds() / 3600
print(hours)


9.712089630277777


5. Based on the results of Question # 4, could you categorize customers into
different categories based on the time period it takes them to visit the
website again? (note: quick assumption)

In [106]:
con.execute("""
WITH with_next AS (
    SELECT
        user_id,
        CAST(event_time AS TIMESTAMP) AS event_time,
        LEAD(CAST(event_time AS TIMESTAMP)) OVER (
            PARTITION BY user_id
            ORDER BY event_time
        ) AS next_visit
    FROM transactions
),
per_user_avg AS (
    SELECT
        user_id,
        AVG(
            EXTRACT(EPOCH FROM next_visit) - EXTRACT(EPOCH FROM event_time)
        ) / 3600.0 AS avg_hours
    FROM with_next
    WHERE next_visit IS NOT NULL
    GROUP BY user_id
)
SELECT
    user_id,
    avg_hours,
    CASE
        WHEN avg_hours < 5 THEN 'Hyperactive'
        WHEN avg_hours >= 5 AND avg_hours < 24 THEN 'Active'
        ELSE 'Not Active'
    END AS activity_level
FROM per_user_avg;
""").df()





FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,user_id,avg_hours,activity_level
0,433167283,0.009167,Hyperactive
1,439042093,0.002778,Hyperactive
2,451539852,16.588210,Active
3,477366532,0.005000,Hyperactive
4,482719593,0.047083,Hyperactive
...,...,...,...
2331039,566196583,0.001806,Hyperactive
2331040,566202525,0.002361,Hyperactive
2331041,566203648,0.010185,Hyperactive
2331042,566216324,0.086806,Hyperactive


6. How many DAU does the E-Commerce website have? Any peaks/troughs?
What about WAU? (you might need to define who is an active user)

In [75]:
con.execute("""
SELECT
    DATE(event_time) AS date,
    COUNT(DISTINCT user_id)  AS dau
FROM transactions
GROUP BY date
ORDER BY date;
""").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,date,dau
0,2019-10-01,190188
1,2019-10-02,184965
2,2019-10-03,170668
3,2019-10-04,209410
4,2019-10-05,194958
5,2019-10-06,193210
6,2019-10-07,186951
7,2019-10-08,221204
8,2019-10-09,214140
9,2019-10-10,205321


In [77]:
con.execute("""
SELECT
    DATE(event_time) AS date,
    COUNT(DISTINCT user_id)  AS dau
FROM transactions
GROUP BY date
ORDER BY dau DESC
LIMIT 5;
""").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,date,dau
0,2019-10-20,231849
1,2019-10-11,230533
2,2019-10-16,230199
3,2019-10-15,230135
4,2019-10-18,229463


In [79]:
con.execute("""
SELECT
    DATE(event_time) AS date,
    COUNT(DISTINCT user_id)  AS dau
FROM transactions
GROUP BY date
ORDER BY dau ASC
LIMIT 5;
""").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,date,dau
0,2019-10-03,170668
1,2019-10-02,184965
2,2019-10-07,186951
3,2019-10-01,190188
4,2019-10-30,191965


In [81]:
con.execute("""
SELECT
    DATE_TRUNC('week', event_time) AS week_start,
    COUNT(DISTINCT user_id) AS wau
FROM transactions
GROUP BY week_start
ORDER BY week_start;
""").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,week_start,wau
0,2019-09-30,851711
1,2019-10-07,1057754
2,2019-10-14,1095864
3,2019-10-21,1089896
4,2019-10-28,633487


7. What is the Average Revenue Per User ARPU and the Average Expected
Revenue Per User in October 2019?
Note: users usually add items to the cart and they can remove it from the cart. The expected
revenue can be estimated to be based on items added to the cart * the likelihood of not
removing it from the cart.

In [86]:
con.execute("""
WITH purchase_revenue AS (
    SELECT
        user_id,
        SUM(price) AS total_revenue
    FROM transactions
    WHERE event_type = 'purchase'
    GROUP BY user_id
)
SELECT AVG(total_revenue) AS ARPU
FROM purchase_revenue;
""").df()

,ARPU
0,662.476455


8. What was the product which was added FIRST to the cart the most?

In [92]:
con.execute("""
WITH first_cart AS (
    SELECT
        user_id,
        product_id,
        ROW_NUMBER() OVER (
            PARTITION BY user_id
            ORDER BY event_time
        ) AS rn
    FROM transactions
    WHERE event_type = 'cart'
)

SELECT
    product_id,
    COUNT(*) AS num_first_added
FROM first_cart
WHERE rn = 1
GROUP BY product_id
ORDER BY num_first_added DESC
LIMIT 1;
""").df()



,product_id,num_first_added
0,1004856,18328
